In [1]:
from schemas import *

In [53]:
df_strech_toll_2025 = pl.read_csv("./data/tables/strechs-tolls-2025.csv", schema_overrides=strechs_tolls_2025_schema)

In [54]:
df_strechs = pl.read_csv("./data/tables/strechs.csv", schema_overrides=strechs_schema)

In [55]:
df_roads = pl.read_csv("./data/tables/roads.csv", schema_overrides=roads_schema)

In [56]:
df_strech_toll_2024 = pl.read_csv("./data/tables/strechs-tolls-2024.csv", schema_overrides=strechs_tolls_2024_schema)

In [65]:
df_strech_toll_2023 = pl.read_csv("./data/tables/strechs-tolls-2023.csv", schema_overrides=strechs_tolls_2023_schema)

In [170]:
df_2025 = df_strech_toll_2025.select(
    "strech_id", "car", "bus-2axle", "bus-3axle", "bus-4axle", "truck-2axle", "truck-3axle", "truck-4axle", "truck-5axle", "truck-6axle"
).rename({"car": "car_2025"})
df_2025 = df_2025.with_columns(bus_2025=pl.mean_horizontal("bus-2axle", "bus-3axle", "bus-4axle")).select(pl.exclude("bus-2axle", "bus-3axle", "bus-4axle"))
df_2025 = df_2025.with_columns(light_truck_2025=pl.mean_horizontal("truck-2axle", "truck-3axle")).select(pl.exclude("truck-2axle", "truck-3axle"))
df_2025 = df_2025.with_columns(
    heavy_truck_2025=pl.mean_horizontal("truck-4axle", "truck-5axle", "truck-6axle")
).select(pl.exclude("truck-4axle", "truck-5axle", "truck-6axle"))

In [169]:
df_2024 = df_strech_toll_2024.select(
    "strech_id", "car", "bus-2axle", "bus-3axle", "bus-4axle", "truck-2axle", "truck-3axle", "truck-4axle", "truck-5axle", "truck-6axle"
).rename({"car": "car_2024"})
df_2024 = df_2024.with_columns(bus_2024=pl.mean_horizontal("bus-2axle", "bus-3axle", "bus-4axle")).select(pl.exclude("bus-2axle", "bus-3axle", "bus-4axle"))
df_2024 = df_2024.with_columns(light_truck_2024=pl.mean_horizontal("truck-2axle", "truck-3axle")).select(pl.exclude("truck-2axle", "truck-3axle"))
df_2024 = df_2024.with_columns(
    heavy_truck_2024=pl.mean_horizontal("truck-4axle", "truck-5axle", "truck-6axle")
).select(pl.exclude("truck-4axle", "truck-5axle", "truck-6axle"))

In [168]:
df_2023 = df_strech_toll_2023.select(
    "strech_id", "car", "bus-2axle", "bus-3axle", "bus-4axle", "truck-2axle", "truck-3axle", "truck-4axle", "truck-5axle", "truck-6axle"
).rename({"car": "car_2023"})
df_2023 = df_2023.with_columns(bus_2023=pl.mean_horizontal("bus-2axle", "bus-3axle", "bus-4axle")).select(pl.exclude("bus-2axle", "bus-3axle", "bus-4axle"))
df_2023 = df_2023.with_columns(light_truck_2023=pl.mean_horizontal("truck-2axle", "truck-3axle")).select(pl.exclude("truck-2axle", "truck-3axle"))
df_2023 = df_2023.with_columns(
    heavy_truck_2023=pl.mean_horizontal("truck-4axle", "truck-5axle", "truck-6axle")
).select(pl.exclude("truck-4axle", "truck-5axle", "truck-6axle"))

In [171]:
df_toll = df_2025.join(df_2024, how="inner", on="strech_id")
df_toll = df_toll.join(df_2023, how="left", on="strech_id")
df_toll = df_toll.join(df_strechs.select("strech_id", "strech_name", "road_id"), on="strech_id")
df_toll = df_toll.join(df_roads.select("road_id", "road_name"), on="road_id", how="left")

In [162]:
df_car_toll = df_toll.filter((pl.col("car_2024") != 0) | (pl.col("car_2023") != 0)).with_columns(
    ((pl.col("car_2025") - pl.col("car_2024")) * 100 / pl.col("car_2024")).alias("inflation_rate_2025"),
    ((pl.col("car_2024") - pl.col("car_2023")) * 100 / pl.col("car_2023")).alias("inflation_rate_2024"),
    ((pl.col("car_2025") / pl.col("car_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [163]:
df_bus_toll = df_toll.filter((pl.col("bus_2024") != 0) | (pl.col("bus_2023") != 0)).with_columns(
    ((pl.col("bus_2025") - pl.col("bus_2024")) * 100 / pl.col("bus_2024")).alias("inflation_rate_2025"),
    ((pl.col("bus_2024") - pl.col("bus_2023")) * 100 / pl.col("bus_2023")).alias("inflation_rate_2024"),
    ((pl.col("bus_2025") / pl.col("bus_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [164]:
df_light_truck_toll = df_toll.filter((pl.col("light_truck_2024") != 0) | (pl.col("light_truck_2023") != 0)).with_columns(
    ((pl.col("light_truck_2025") - pl.col("light_truck_2024")) * 100 / pl.col("light_truck_2024")).alias("inflation_rate_2025"),
    ((pl.col("light_truck_2024") - pl.col("light_truck_2023")) * 100 / pl.col("light_truck_2023")).alias("inflation_rate_2024"),
    ((pl.col("light_truck_2025") / pl.col("light_truck_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [172]:
df_heavy_truck_toll = df_toll.filter((pl.col("heavy_truck_2024") != 0) | (pl.col("heavy_truck_2023") != 0)).with_columns(
    ((pl.col("heavy_truck_2025") - pl.col("heavy_truck_2024")) * 100 / pl.col("heavy_truck_2024")).alias("inflation_rate_2025"),
    ((pl.col("heavy_truck_2024") - pl.col("heavy_truck_2023")) * 100 / pl.col("heavy_truck_2023")).alias("inflation_rate_2024"),
    ((pl.col("heavy_truck_2025") / pl.col("heavy_truck_2023") - 1) * 100).alias("inflation_rate_2023_2025")
).filter(pl.col("inflation_rate_2023_2025").is_not_null())

In [165]:
df_car_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "car_2025", "car_2024", "car_2023", "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_car.csv")

In [166]:
df_bus_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "bus_2025", "bus_2024", "bus_2023", "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_bus.csv")

In [167]:
df_light_truck_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "light_truck_2025", "light_truck_2024", "light_truck_2023", 
    "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_light_truck.csv")

In [173]:
df_heavy_truck_toll.sort("inflation_rate_2023_2025", descending=True).select(
    "strech_id", "heavy_truck_2025", "heavy_truck_2024", "heavy_truck_2023", 
    "strech_name", "road_name", "inflation_rate_2025", "inflation_rate_2024", "inflation_rate_2023_2025"
).write_csv("inflation_rate_heavy_truck.csv")

In [109]:
df_car_toll.select(pl.mean("inflation_rate_2025", "inflation_rate_2024"))

inflation_rate_2025,inflation_rate_2024
f32,f32
5.674502,3.483997


In [120]:
df_car_toll.select(pl.col("inflation_rate_2025")).filter(pl.col("inflation_rate_2025") > 4.5).count() / df_car_toll.height

inflation_rate_2025
f64
0.735818


In [122]:
df_car_toll.select(pl.col("cum_inflation_rate")).filter(pl.col("cum_inflation_rate") > 9.9).count() / df_car_toll.height

cum_inflation_rate
f64
0.281199
